In [3]:
import json
import numpy as np
import pandas as pd
import string
import re
import nltk

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
df = pd.read_json('12147_final_df.json', lines=True)

In [ ]:
y = df['match_seed_urlh']
df.drop(['match_seed_urlh'], axis=1)

In [11]:
## train test split - 80:20
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

In [12]:
X_train['match_seed_urlh'] = y

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [197]:
X_train.to_csv('DTrain_80.csv', index=False)
X_test.to_csv('DTest_20.csv', index=False)

#### Read train data

In [ ]:
X_train = pd.read_csv('DTrain_80.csv')

In [120]:
corpus = X_train[['title', 'match_title', 'description', 'match_description', 'color']]

In [121]:
corpus.head()

,title,match_title,description,match_description,color
107867,Theory Silk Shell Top IVORY Small---,Theory Silk Shell Top IVORY Medium---,,,IVORY
366665,Chantelle Lingerie C Magnifique Seamless Unlin...,"C Magnifique Minimizer Bra 1891, Online Only I...",Designed to minimize your bustline by one inch...,Full-coverage support with a delightful subtle...,IVORY
135770,RAG AND BONE rag & bone Shiloh Combat Boot (Wo...,"rag & bone Shiloh Boot | The Style Room, power...","Side panels lend a bit of give to tough, comba...",,BLACK
15283,Bixbee Kitty Water Resistant Lunchbox (Kids) P...,Bixbee Lunchbox Animal,The face of a pretty kitty defines a playful w...,,Pink
66925,Maison Francis Kurkdjian Paris Grand Soir Eau ...,Maison Francis Kurkdjian 6.8 oz. Grand Soir Ea...,<strong>What it is</strong>: A sophisticated a...,,No Color


In [ ]:
## applying lowercase function to text columns

corpus['title'] = corpus['title'].str.lower()
corpus['match_title'] = corpus['match_title'].str.lower()
corpus['description'] = corpus['description'].str.lower()
corpus['match_description'] = corpus['match_description'].str.lower()
corpus['color'] = corpus['color'].str.lower()

In [ ]:
## replacing '&' with 'and' keyword to conserve semantic meaning of sentences

corpus['title'] = corpus['title'].str.replace('&','and')
corpus['match_title'] = corpus['match_title'].str.replace('&','and')
corpus['description'] = corpus['description'].str.replace('&','and')
corpus['match_description'] = corpus['match_description'].str.replace('&','and')
corpus['color'] = corpus['color'].str.replace('&','and')

In [ ]:
## removing html-tags from data

corpus['title'] = corpus['title'].str.replace(r'<[^<>]*>', '', regex=True)
corpus['match_title'] = corpus['match_title'].str.replace(r'<[^<>]*>', '', regex=True)
corpus['description'] = corpus['description'].str.replace(r'<[^<>]*>', '', regex=True)
corpus['match_description'] = corpus['match_description'].str.replace(r'<[^<>]*>', '', regex=True)
corpus['color'] = corpus['color'].str.replace(r'<[^<>]*>', '', regex=True)

In [125]:
corpus['description'] = corpus['description'].str.replace(r'what it is:', '')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
## removing special chars => !"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`
import string
import re

PUNCT_TO_REMOVE = string.punctuation
PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace("-","")
# PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace(".","")
PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace("&","")

def remove_special_chars(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [ ]:
## making some values as str to solve "NaN" value issue

corpus['title'] = corpus['title'].astype(str)
corpus['match_title'] = corpus['match_title'].astype(str)
corpus['description'] = corpus['description'].astype(str)
corpus['match_description'] = corpus['match_description'].astype(str)
corpus['color'] = corpus['color'].astype(str)

In [ ]:
## removing unwanted chars

corpus["title"] = corpus["title"].apply(lambda text: remove_special_chars(text))
corpus["match_title"] = corpus["match_title"].apply(lambda text: remove_special_chars(text))
corpus["description"] = corpus["description"].apply(lambda text: remove_special_chars(text))
corpus["match_description"] = corpus["match_description"].apply(lambda text: remove_special_chars(text))
corpus["color"] = corpus["color"].apply(lambda text: remove_special_chars(text))

In [ ]:
## removing numbers from text strings

corpus['title'] = corpus['title'].str.replace('\d+', '')
corpus['match_title'] = corpus['match_title'].str.replace('\d+', '')
corpus['description'] = corpus['description'].str.replace('\d+', '')
corpus['match_description'] = corpus['match_description'].str.replace('\d+', '')
corpus['color'] = corpus['color'].str.replace('\d+', '')

In [ ]:
## replacing '-' with ' ' whitespace to properly seperate words

corpus['title'] = corpus['title'].str.replace('-',' ')
corpus['match_title'] = corpus['match_title'].str.replace('-',' ')             
corpus['description'] = corpus['description'].str.replace('-',' ')
corpus['match_description'] = corpus['match_description'].str.replace('-',' ')
corpus['color'] = corpus['color'].str.replace('-',' ')

In [134]:
corpus.to_csv('processed_Dtrain_4f.csv', index=False)

In [ ]:
# dtype={'title': object, 'match_title':object, 'description':object, 'match_description':object, 'color':object}

In [250]:
corpus = pd.read_csv('processed_Dtrain_4f.csv')

In [251]:
corpus['title'] = [ '' if x is np.NaN else x for x in corpus['title'] ]
corpus['match_title'] = [ '' if x is np.NaN else x for x in corpus['match_title'] ]
corpus['description'] = [ '' if x is np.NaN else x for x in corpus['description'] ]
corpus['match_description'] = [ '' if x is np.NaN else x for x in corpus['match_description'] ]
corpus['color'] = [ '' if x is np.NaN else x for x in corpus['color'] ]

In [252]:
## performing tokenization - breaking sentences into lexical units

import nltk
from nltk.tokenize import word_tokenize

In [253]:
corpus['title'] = corpus.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)
corpus['match_title'] = corpus.apply(lambda row: nltk.word_tokenize(row['match_title']), axis=1)
corpus['description'] = corpus.apply(lambda row: nltk.word_tokenize(row['description']), axis=1)
corpus['match_description'] = corpus.apply(lambda row: nltk.word_tokenize(row['match_description']), axis=1)
corpus['color'] = corpus.apply(lambda row: nltk.word_tokenize(row['color']), axis=1)

In [48]:
corpus.to_csv('processed_Dtrain_4f_tokenized.csv', index=False)

### training on 'title' feature only

In [254]:
sentences = []
for row in corpus['title']:
    sentences.append(list(row))

In [256]:
from gensim.models import Word2Vec
model = Word2Vec(sentences, min_count=1, size=200, workers=6, window=5, iter=10)

In [257]:
len(model.wv.vocab)

26698

In [258]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.699643611907959),
 ('coat', 0.652625560760498),
 ('parka', 0.6479331254959106),
 ('raincoat', 0.6120079755783081),
 ('miniskirt', 0.5529841184616089),
 ('pullover', 0.507602870464325),
 ('cardigan', 0.4950186014175415),
 ('joggers', 0.4948863983154297),
 ('leggings', 0.491107702255249),
 ('hoodie', 0.4691888988018036)]

In [259]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('shoe', 0.5678728818893433),
 ('loafer', 0.5505174398422241),
 ('slide', 0.5277519822120667),
 ('sandal', 0.5206455588340759),
 ('boot', 0.5197134613990784),
 ('flop', 0.5013018846511841),
 ('gum', 0.48239028453826904),
 ('bootie', 0.48095059394836426),
 ('flip', 0.4711340069770813),
 ('slipper', 0.45892223715782166)]

In [264]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('planner', 0.6576274037361145),
 ('grabber', 0.6383662223815918),
 ('toolbox', 0.6313226819038391),
 ('hedgehog', 0.6312799453735352),
 ('teething', 0.626238226890564),
 ('landspeeder™', 0.6118741631507874),
 ('blankie', 0.6074351072311401),
 ('pedestal', 0.6036496162414551),
 ('ark', 0.5969358682632446),
 ('silverlite', 0.5964605808258057)]

In [270]:
## Change of hyper-parameters (windows size= (5->7))
model = Word2Vec(sentences, min_count=1, size=200, workers=6, window=7, iter=10)

In [271]:
len(model.wv.vocab)

26698

In [273]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.7439987659454346),
 ('coat', 0.6748998165130615),
 ('parka', 0.6622467637062073),
 ('raincoat', 0.5972958207130432),
 ('miniskirt', 0.5470995306968689),
 ('cardigan', 0.5419610738754272),
 ('joggers', 0.5329744815826416),
 ('overshirt', 0.5227255821228027),
 ('pullover', 0.52142733335495),
 ('leggings', 0.5188730359077454)]

In [277]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('shoe', 0.5731929540634155),
 ('sandal', 0.5446249842643738),
 ('slide', 0.5439327955245972),
 ('loafer', 0.5302315950393677),
 ('flop', 0.5233615636825562),
 ('boot', 0.5102553367614746),
 ('bootie', 0.5006389617919922),
 ('slipper', 0.4761015772819519),
 ('moccasin', 0.46150079369544983),
 ('flip', 0.4588810205459595)]

In [278]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('grabber', 0.6708623766899109),
 ('hedgehog', 0.6637697815895081),
 ('puppy', 0.6562122106552124),
 ('teething', 0.6214553117752075),
 ('programmer', 0.6004025340080261),
 ('minicall', 0.5978298783302307),
 ('planner', 0.5958477258682251),
 ('ukelele', 0.5952675342559814),
 ('toolbox', 0.5876369476318359),
 ('peachlite', 0.5782372951507568)]

In [283]:
## Change of hyper-parameters (windows size= (7->3))
model = Word2Vec(sentences, min_count=1, size=200, workers=6, window=3, iter=10)

In [284]:
len(model.wv.vocab)

26698

In [285]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.6460692882537842),
 ('parka', 0.6156010627746582),
 ('coat', 0.6086779832839966),
 ('raincoat', 0.59493088722229),
 ('bib', 0.498401403427124),
 ('miniskirt', 0.49246296286582947),
 ('leggings', 0.48663175106048584),
 ('gloves', 0.46884340047836304),
 ('pullover', 0.46256542205810547),
 ('joggers', 0.4618302583694458)]

In [286]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('shoe', 0.575181782245636),
 ('loafer', 0.5149010419845581),
 ('flip', 0.4916948676109314),
 ('flop', 0.48971760272979736),
 ('sandal', 0.48794713616371155),
 ('boot', 0.4714520573616028),
 ('gum', 0.4624810218811035),
 ('slide', 0.4525415003299713),
 ('slipper', 0.4499064087867737),
 ('elk', 0.41835683584213257)]

In [287]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('planner', 0.6667201519012451),
 ('programmer', 0.6560723781585693),
 ('instruments', 0.6409258246421814),
 ('pearlgold', 0.6335291862487793),
 ('landspeeder™', 0.6239038705825806),
 ('fucshia', 0.6208600997924805),
 ('ukelele', 0.6207716464996338),
 ('resisting', 0.6168082356452942),
 ('tidbit', 0.6146268844604492),
 ('clubhouse', 0.6140837073326111)]

In [289]:
## Change hyperparamter (windows size= (3->9)
model = Word2Vec(sentences, min_count=1, size=200, workers=6, window=9, iter=10)

In [291]:
len(model.wv.vocab)

26698

In [292]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.7762060165405273),
 ('parka', 0.6845677495002747),
 ('coat', 0.6841486096382141),
 ('raincoat', 0.6209619045257568),
 ('joggers', 0.581281304359436),
 ('cardigan', 0.550774097442627),
 ('miniskirt', 0.5494042634963989),
 ('pullover', 0.548337996006012),
 ('leggings', 0.5479260683059692),
 ('sweatpants', 0.5445652604103088)]

In [293]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('shoe', 0.5893261432647705),
 ('sandal', 0.5477840900421143),
 ('slide', 0.5447310209274292),
 ('loafer', 0.5409703850746155),
 ('boot', 0.5202202796936035),
 ('flop', 0.4905245006084442),
 ('bootie', 0.48400360345840454),
 ('clog', 0.4798380732536316),
 ('slipper', 0.47803324460983276),
 ('gargoyle', 0.47009795904159546)]

In [294]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('grabber', 0.5979047417640686),
 ('cradle', 0.5970484018325806),
 ('puppy', 0.5957483053207397),
 ('toolbox', 0.5934377312660217),
 ('hedgehog', 0.592780590057373),
 ('programmer', 0.5900658369064331),
 ('blankie', 0.5878332853317261),
 ('ark', 0.5858703851699829),
 ('ukelele', 0.581758439540863),
 ('bola', 0.5698684453964233)]

In [296]:
model.most_similar('charger')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('oblio', 0.7344127893447876),
 ('speaker', 0.6161797046661377),
 ('pods', 0.6031231880187988),
 ('lexon', 0.5906018018722534),
 ('terrazzo', 0.5842087268829346),
 ('terazzo', 0.5829258561134338),
 ('bluetooth®', 0.5524916648864746),
 ('earbuds', 0.5517601370811462),
 ('lansing', 0.5196601748466492),
 ('cleaner', 0.5190761089324951)]

In [298]:
## Change hyperparamter: iter (10->30)
model = Word2Vec(sentences, min_count=1, size=200, workers=6, window=9, iter=30)

In [299]:
len(model.wv.vocab)

26698

In [300]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.7265077233314514),
 ('coat', 0.6830401420593262),
 ('parka', 0.6362659335136414),
 ('raincoat', 0.5556483864784241),
 ('pullover', 0.5439473986625671),
 ('joggers', 0.5421296954154968),
 ('hoodie', 0.5406759977340698),
 ('pants', 0.5297666788101196),
 ('cardigan', 0.5138182044029236),
 ('sweatshirt', 0.5048598051071167)]

In [301]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sandal', 0.6780663132667542),
 ('shoe', 0.6225469708442688),
 ('boot', 0.594718337059021),
 ('loafer', 0.5941756963729858),
 ('slide', 0.5841199159622192),
 ('bootie', 0.5822051167488098),
 ('flop', 0.5315537452697754),
 ('flip', 0.5250399112701416),
 ('moccasin', 0.5183849334716797),
 ('flat', 0.5160670280456543)]

In [302]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cottesloe', 0.5120266079902649),
 ('grabber', 0.5072698593139648),
 ('teething', 0.4988667368888855),
 ('multco', 0.47503307461738586),
 ('programmer', 0.4733580946922302),
 ('blankie', 0.4664655327796936),
 ('allix', 0.4608061909675598),
 ('ark', 0.4499088227748871),
 ('flashcards', 0.445121169090271),
 ('ukelele', 0.439644455909729)]

In [307]:
## Changing embedding dimension 200->300
model = Word2Vec(sentences, min_count=1, size=300, workers=6, window=9, iter=30)

In [308]:
len(model.wv.vocab)

26698

In [309]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.7053204774856567),
 ('coat', 0.6617432832717896),
 ('parka', 0.6214284896850586),
 ('raincoat', 0.5477792024612427),
 ('joggers', 0.5271948575973511),
 ('pants', 0.5196796655654907),
 ('pullover', 0.5123668909072876),
 ('hoodie', 0.5090541839599609),
 ('sweatpants', 0.5053430795669556),
 ('cardigan', 0.4997313618659973)]

In [310]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sandal', 0.6437646150588989),
 ('shoe', 0.5988448262214661),
 ('boot', 0.5933150053024292),
 ('loafer', 0.5691571235656738),
 ('bootie', 0.5663340091705322),
 ('slide', 0.5650060176849365),
 ('flop', 0.546144962310791),
 ('slipper', 0.5213775634765625),
 ('flat', 0.5031276345252991),
 ('flip', 0.4806806743144989)]

In [312]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('grabber', 0.4878574311733246),
 ('allix', 0.482843279838562),
 ('goal', 0.47212713956832886),
 ('sophiesticated', 0.4712635576725006),
 ('blankie', 0.46532467007637024),
 ('multco', 0.4456629157066345),
 ('quarterly', 0.4438058137893677),
 ('ark', 0.44104254245758057),
 ('programmer', 0.43792349100112915),
 ('cottesloe', 0.4276331663131714)]

In [314]:
## Change in hyperparameter: window (9-7)
model = Word2Vec(sentences, min_count=1, size=300, workers=6, window=7, iter=30)

In [316]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.6921069025993347),
 ('coat', 0.6389036178588867),
 ('parka', 0.5980799198150635),
 ('raincoat', 0.5334936380386353),
 ('pants', 0.49985814094543457),
 ('pullover', 0.4899914264678955),
 ('joggers', 0.48584848642349243),
 ('cardigan', 0.4743598699569702),
 ('leggings', 0.4682120680809021),
 ('sweatpants', 0.46053311228752136)]

In [317]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sandal', 0.6327903866767883),
 ('shoe', 0.5975995063781738),
 ('loafer', 0.5783309936523438),
 ('boot', 0.5776581168174744),
 ('slide', 0.5721780061721802),
 ('bootie', 0.5647616386413574),
 ('flop', 0.5460494756698608),
 ('slipper', 0.505042314529419),
 ('flip', 0.5047544240951538),
 ('flat', 0.48346245288848877)]

In [318]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('goal', 0.5091447234153748),
 ('blankie', 0.5063661336898804),
 ('teething', 0.4934045374393463),
 ('programmer', 0.4832916557788849),
 ('puppy', 0.4738975167274475),
 ('grabber', 0.4737568497657776),
 ('flashcards', 0.46878406405448914),
 ('planner', 0.46817195415496826),
 ('quarterly', 0.46658653020858765),
 ('clovelly', 0.45564427971839905)]

In [320]:
## Changing mincount (1-2)
model = Word2Vec(sentences, min_count=2, size=200, workers=6, window=9, iter=30)

In [321]:
len(model.wv.vocab)

22952

In [322]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.7161762714385986),
 ('coat', 0.674823522567749),
 ('parka', 0.6393566131591797),
 ('raincoat', 0.5564126372337341),
 ('joggers', 0.551395058631897),
 ('pullover', 0.5374972224235535),
 ('pants', 0.5299801826477051),
 ('hoodie', 0.5285268425941467),
 ('sweatpants', 0.5223351120948792),
 ('cardigan', 0.5125088691711426)]

In [324]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sandal', 0.6674016714096069),
 ('shoe', 0.6106605529785156),
 ('slide', 0.586976945400238),
 ('boot', 0.5850046873092651),
 ('bootie', 0.5841420292854309),
 ('loafer', 0.5797597169876099),
 ('flop', 0.5296533107757568),
 ('slipper', 0.5254599452018738),
 ('flat', 0.5140346884727478),
 ('flip', 0.5092979669570923)]

In [325]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('grabber', 0.5308519601821899),
 ('multco', 0.5242213010787964),
 ('cottesloe', 0.5116217136383057),
 ('ark', 0.49865520000457764),
 ('blankie', 0.4929807186126709),
 ('noahs', 0.4885956048965454),
 ('allix', 0.4647219479084015),
 ('sophiesticated', 0.4602026641368866),
 ('roffe', 0.45898887515068054),
 ('therapeutic', 0.45886945724487305)]

### training on all features - (title, match_title, description, min_description, color)

In [328]:
sentences = []
for row in corpus['title']:
    sentences.append(list(row))
    
for row in corpus['match_title']:
    sentences.append(list(row))

for row in corpus['description']:
    sentences.append(list(row))
    
for row in corpus['match_description']:
    sentences.append(list(row))
    
for row in corpus['color']:
    sentences.append(list(row))

In [329]:
len(sentences)

1540205

In [333]:
model = Word2Vec(sentences, min_count=1, size=200, workers=6, window=9, iter=30)

In [335]:
len(model.wv.vocab)

89813

In [336]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.741928219795227),
 ('parka', 0.6525267362594604),
 ('raincoat', 0.6182602643966675),
 ('cardigan', 0.6044610738754272),
 ('hoodie', 0.5461168885231018),
 ('windbreaker', 0.5416706800460815),
 ('anorak', 0.5351794958114624),
 ('blazer', 0.5113333463668823),
 ('sweatshirt', 0.5003517866134644),
 ('gilet', 0.4896860122680664)]

In [337]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sneakers', 0.7442237734794617),
 ('sandal', 0.6258714199066162),
 ('bootie', 0.617617666721344),
 ('shoe', 0.6070288419723511),
 ('loafer', 0.5863436460494995),
 ('boot', 0.562466561794281),
 ('mule', 0.5350886583328247),
 ('derby', 0.4969525933265686),
 ('trainers', 0.4875684976577759),
 ('shoes', 0.4820402264595032)]

In [338]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('coding', 0.38513898849487305),
 ('recieve', 0.3376978039741516),
 ('sensesaromatherapy', 0.3370952308177948),
 ('nokidhungryorgonedollar', 0.33002710342407227),
 ('haba', 0.3283977508544922),
 ('ding', 0.3274790644645691),
 ('beginninghow', 0.32530075311660767),
 ('baubles', 0.32338595390319824),
 ('pacifiers', 0.3120989203453064),
 ('timex™', 0.3120735287666321)]

In [339]:
model.most_similar('sofa')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('couch', 0.5190832614898682),
 ('marinière', 0.402392715215683),
 ('torchlight', 0.38560929894447327),
 ('chilling', 0.38253551721572876),
 ('comfiest', 0.3780391216278076),
 ('occassion', 0.36806732416152954),
 ('go', 0.3645569980144501),
 ('plié', 0.36319711804389954),
 ('btch', 0.362680584192276),
 ('gowho', 0.35397690534591675)]

In [340]:
model.most_similar('charger')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('wi', 0.4996408224105835),
 ('adapter', 0.4524473249912262),
 ('oblio', 0.4518716335296631),
 ('incompatible', 0.44209590554237366),
 ('aaa', 0.4414590001106262),
 ('keysmart', 0.44033461809158325),
 ('wifi', 0.43954890966415405),
 ('assistant', 0.43842485547065735),
 ('android', 0.4331709146499634),
 ('canceling', 0.4292817711830139)]

### changing the hyperparameters

In [342]:
model = Word2Vec(sentences, min_count=1, size=200, workers=6, window=7, iter=30)

In [343]:
len(model.wv.vocab)

89813

In [344]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.7360107898712158),
 ('parka', 0.6576131582260132),
 ('raincoat', 0.6079296469688416),
 ('cardigan', 0.5949206948280334),
 ('anorak', 0.554686427116394),
 ('hoodie', 0.5396915674209595),
 ('blazer', 0.5189775824546814),
 ('windbreaker', 0.5032904148101807),
 ('sweater', 0.47806912660598755),
 ('sweatshirt', 0.476653516292572)]

In [345]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sneakers', 0.7375390529632568),
 ('bootie', 0.6308452486991882),
 ('sandal', 0.627427339553833),
 ('shoe', 0.6104562878608704),
 ('loafer', 0.5932319760322571),
 ('boot', 0.5632538199424744),
 ('mule', 0.551547646522522),
 ('derby', 0.5009572505950928),
 ('shoes', 0.48454520106315613),
 ('trainers', 0.46670132875442505)]

In [346]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hedgehog', 0.35714003443717957),
 ('apps', 0.3535084128379822),
 ('bathtub', 0.3454494774341583),
 ('cakes', 0.3265300989151001),
 ('podcasts', 0.3219749629497528),
 ('coding', 0.32107093930244446),
 ('charges', 0.3204496502876282),
 ('touchbar', 0.3149166405200958),
 ('bandages', 0.3136991858482361),
 ('brutal', 0.3134861886501312)]

In [347]:
model.most_similar('sofa')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('couch', 0.5608115792274475),
 ('plié', 0.43403780460357666),
 ('barre', 0.4010542333126068),
 ('goshade', 0.363606333732605),
 ('marinière', 0.36343634128570557),
 ('sole—for', 0.3599902391433716),
 ('go', 0.359938383102417),
 ('tortillas', 0.3587237000465393),
 ('snuggling', 0.3564422130584717),
 ('streets', 0.35636264085769653)]

In [348]:
model.most_similar('charger')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('wi', 0.45451295375823975),
 ('adapter', 0.43053901195526123),
 ('keysmart', 0.42842555046081543),
 ('soundsport®', 0.4195632338523865),
 ('device', 0.4154983162879944),
 ('phone', 0.4125204086303711),
 ('assistant', 0.4110783040523529),
 ('oblio', 0.41051608324050903),
 ('qi', 0.40727007389068604),
 ('electronics', 0.4071753919124603)]

In [350]:
## Changing hyperparameter: dim-size (200-300)
model = Word2Vec(sentences, min_count=1, size=300, workers=6, window=9, iter=30)

In [352]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.6987109184265137),
 ('parka', 0.5827820301055908),
 ('cardigan', 0.5420409440994263),
 ('raincoat', 0.524449348449707),
 ('hoodie', 0.5065807104110718),
 ('windbreaker', 0.4877925515174866),
 ('anorak', 0.4775785207748413),
 ('sweatshirt', 0.45647621154785156),
 ('blazer', 0.45554202795028687),
 ('sweater', 0.44973719120025635)]

In [353]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sneakers', 0.7191941738128662),
 ('bootie', 0.5944501161575317),
 ('sandal', 0.5931066274642944),
 ('shoe', 0.5624397397041321),
 ('loafer', 0.5511095523834229),
 ('boot', 0.5401980876922607),
 ('mule', 0.4582121968269348),
 ('derby', 0.4527275562286377),
 ('shoes', 0.42743754386901855),
 ('trainers', 0.41453176736831665)]

In [354]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('skidproof', 0.3245064616203308),
 ('beginninghow', 0.3093853294849396),
 ('valleys', 0.29352158308029175),
 ('baubles', 0.2912706732749939),
 ('tent', 0.28146907687187195),
 ('apps', 0.28036925196647644),
 ('bathtub', 0.27838143706321716),
 ('ding', 0.27599817514419556),
 ('appealwhy', 0.27366405725479126),
 ('inches•', 0.2688619792461395)]

In [355]:
model.most_similar('sofa')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('couch', 0.4735751748085022),
 ('goshade', 0.388747900724411),
 ('plié', 0.38663190603256226),
 ('chilling', 0.35262930393218994),
 ('logo—based', 0.3328953683376312),
 ('btch', 0.32912561297416687),
 ('snuggling', 0.3253437280654907),
 ('biceps', 0.32035568356513977),
 ('bitch', 0.32014116644859314),
 ('irons—is', 0.31433945894241333)]

In [356]:
model.most_similar('charger')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('soundsport®', 0.42741966247558594),
 ('wi', 0.4097926914691925),
 ('aaa', 0.3956053555011749),
 ('cleantray', 0.3732897937297821),
 ('device', 0.366916298866272),
 ('charging', 0.36455193161964417),
 ('planter', 0.36025023460388184),
 ('incompatible', 0.3600127398967743),
 ('usb', 0.35808759927749634),
 ('soundbar', 0.35481107234954834)]

In [358]:
## Changing hyperparameter (mincount 1-2)
model = Word2Vec(sentences, min_count=2, size=200, workers=6, window=9, iter=30)

In [359]:
len(model.wv.vocab)

68207

In [360]:
model.most_similar('jacket')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('vest', 0.7335571050643921),
 ('parka', 0.6453668475151062),
 ('raincoat', 0.6063073873519897),
 ('cardigan', 0.5994545221328735),
 ('hoodie', 0.5367416143417358),
 ('windbreaker', 0.5242978930473328),
 ('blazer', 0.513157069683075),
 ('anorak', 0.5049210786819458),
 ('snowsuit', 0.4945971667766571),
 ('sweatshirt', 0.4891900420188904)]

In [361]:
model.most_similar('sneaker')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sneakers', 0.7455086708068848),
 ('sandal', 0.6187825202941895),
 ('shoe', 0.6066161394119263),
 ('bootie', 0.6053400635719299),
 ('loafer', 0.5857114791870117),
 ('boot', 0.5588629245758057),
 ('mule', 0.5228941440582275),
 ('derby', 0.48803332448005676),
 ('shoes', 0.4660491347312927),
 ('slipper', 0.4418397843837738)]

In [362]:
model.most_similar('mobile')

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('bathtub', 0.3337714672088623),
 ('touchbar', 0.33204734325408936),
 ('comotomo', 0.33136212825775146),
 ('cakes', 0.3232908844947815),
 ('bills', 0.3195013999938965),
 ('girlfriends', 0.31939882040023804),
 ('slice', 0.30698084831237793),
 ('baubles', 0.3041940927505493),
 ('greatness', 0.3038279414176941),
 ('farmer', 0.30352920293807983)]

In [363]:
model.most_similar('sofa') 

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('couch', 0.5766681432723999),
 ('marinière', 0.3916207253932953),
 ('plié', 0.3886525630950928),
 ('goshade', 0.3551502227783203),
 ('comfiest', 0.3533968925476074),
 ('snuggling', 0.3502125144004822),
 ('torchlight', 0.3469448387622833),
 ('chilling', 0.3460928499698639),
 ('gowho', 0.3422125577926636),
 ('taxis', 0.33958202600479126)]

In [364]:
model.most_similar('charger') 

C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('aaa', 0.4739176034927368),
 ('wi', 0.4678841829299927),
 ('usb', 0.4677230715751648),
 ('assistant', 0.45538416504859924),
 ('earbuds', 0.4379603862762451),
 ('battery', 0.43377041816711426),
 ('rechargeable', 0.4254394471645355),
 ('incompatible', 0.4230232834815979),
 ('speaker', 0.4202727675437927),
 ('android', 0.40753012895584106)]

In [1]:
import json

data_ = []
for line in open('12147_final_df.json', 'r'):
    data_.append(json.loads(line))

In [2]:
data_[4456]

{'index': 7325.0,
 'match_available_price': -1.0,
 'match_source': 'Amazon-US',
 'match_stock': 'Out of Stock',
 'match_crawl_time': 20210228162,
 'match_raw_subcategory': 'Boots',
 'match_seed_urlh': '5efb4ac2212f109ebc889dc2a735f791dfc4119a',
 'match_mrp': -1.0,
 'match_title': "UGG Kids' Classic Clear Mini Ii Fashion Boot",
 'match_raw_brand': '',
 'match_urlh': 'aae97172ffc6c29bc80b6e9eaaa1db65044ce689',
 'match_sku': 'B08LLYWTGQ',
 'match_raw_category': 'Clothing, Shoes & Jewelry',
 'match_color': 'unknown',
 'match_crawl_type': 'product',
 'match_crawl_date': 20210228,
 'match_status': 0.0,
 'match_url': 'https://www.amazon.com/UGG-Kids-Classic-Clear-Sulfur/dp/B0854QJSX4',
 'match_datasource_s': '',
 'match_country': 'usa',
 'match_size': '6 Big Kid',
 'match_upc': '',
 'match_seed_url': 'na',
 'match_thumbnail': 'https://images-na.ssl-images-amazon.com/images/I/41FdHfM1roL._AC_.jpg',
 'match_timestamp': '2021-02-28T17:53:15.112Z',
 'match__version_': 1.692962437e+18,
 'match_raw

In [5]:
df = pd.read_json('12147_final_df.json', lines=True)

In [8]:
df['match_type'].value_counts()

valid      332208
invalid     52844
Name: match_type, dtype: int64

In [9]:
df['task_type'].value_counts()

Exact Matching    369539
Name: task_type, dtype: int64